# An introduction to Transformers

This presentation is heavily inspired by [Jay Alammar](http://jalammar.github.io/) blog.

## Introduction

* Transformers use the attention as a means to parallelization
* Proposed in the famous paper: "Attention is All You Need"
* Implemented in many NLP and ML frameworks including Tensorflow and PyTorch

**Here is how a trained transformer model works for the task of machine translation**


<img src="http://jalammar.github.io/images/t/the_transformer_3.png" alt="Transformer translation" width="80%"/>

**Let's take a look inside the black box**


<img src="http://jalammar.github.io/images/t/The_transformer_encoders_decoders.png" alt="Transformer structure" width="80%"/>

**An even deeper look inside each building block**


<img src="http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" alt="Transformer encoder and decoder" width="80%"/>

**Now let's see how each encoder is constructed**


<img src="http://jalammar.github.io/images/t/Transformer_encoder.png" alt="Encoder structure" width="80%"/>

* The input to each encoder first goes through a Self-Attention layer
* The self-attention layer implements the idea of attention, taking other word's importance to account
* The output of the self-attention layer goes through a FFNN

**Now let's see how each decoder is constructed**


<img src="http://jalammar.github.io/images/t/Transformer_decoder.png" alt="Decoder structure" width="80%"/>

* Similar to the Encoder structure with a Encoder-Decoder Attention layer in between
* The extra layer helps the decoder to focus on more relevant parts

We will see how each of these building blocks work.

### Encoder

No surprise, we need to first embed the words into numerical vectors. As we also did with the `seq2seq` model.

Let's consider the following embedding for the input text.

<img src="http://jalammar.github.io/images/t/embeddings.png" alt="Embedding vector" width="80%"/>

The first encoder on the bottom receives the embedding vectors and the rest just receive the output of the previous encoders in the stack.

**Now let's see what happens at the bottom-most encoder**


<img src="http://jalammar.github.io/images/t/encoder_with_tensors.png" alt="First encoder" width="80%"/>

Note that the self attention layer is there to incorporate other words' importance into account when computing the output vector for each word. So there are dependencies and we can't have parallel execution. However, in the FFNN there is no depdencies and it executes in a parallel mode.

**Past the first encoder, let's engage with upcoming encoders**


<img src="http://jalammar.github.io/images/t/encoder_with_tensors_2.png" alt="Other encoder" width="80%"/>

#### Self Attention

In the following sentence, what does the "it" refers to? It's an easy question for us humans, but what about computers?

*The animal didn't cross the street because it was too tired.*

The purpose of self-attention layer is to help associating the word *it* with the word *animal*. This is similar to what RNNs do with the `hidden state`, i.e. incorporating the importance and hence, vector representation, of other previous words when processing the current word.

As we can see, the word *it* is mostly associated with *the animal*

<img src="http://jalammar.github.io/images/t/transformer_self-attention_visualization.png" alt="self-attention vector" width="80%"/>

Now let's dig deeper into the process of self-attention layer.

Given x<sub>1</sub> and x<sub>2</sub> as the embedded vectors for the inputs to an encoder, for each of them three vectors are computed:
1. Query vector
2. Key vector
3. Value vector

They are computed by multiplying the input vector by their corresponding matrices, W<sup>Q</sup>, W<sup>K</sup> and W<sup>V</sup>.

Note that the mentioned matrices are **trained** by the model, whereas the input vector is given.

**Query, Key, and Value vectors computation using the input vectors and the trained vectors W**

<img src="http://jalammar.github.io/images/t/transformer_self_attention_vectors.png" alt="Q K and V vectors computation" width="80%"/>

As the next step, q<sub>i</sub> and k<sub>i</sub> vectors are dot producted, the resulted value is divided by the square root of k<sub>i</sub> vector dimension (defaults to 8 in the original paper), then goes through a Softmax, and finally multiplied by the v<sub>i</sub> vector to compute **current word's** value vector. The **attention** vector for the current word is the summation of all value vectors.

<img src="http://jalammar.github.io/images/t/self-attention-output.png" alt="attention vector computation" width="80%"/>

* The score for each word is calculated by the dot product of query and key vectors.
* The computed score goes through the square root and the softmax, for the purpose of normalization
* The softmax determines the importance of each word at the **current word position**
* Each word is the most important, hence the highest softmax value, at its own position
* The softmax score is multiplied by the value vector to amplify/drown out the imortance of **every word** at the **current word position**
* Finally, all value vectors are summed up to compute the **attention vector** for the **current word position**

**Seems like a lot of computation? Well, linear algebra is here to help us!**

<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation.png" alt="attention vector matrix computation" width="80%"/>

**And then**

<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" alt="attention vector matrix computation 2" width="80%"/>

#### Some leftover details...

**How does the transformer know about the position of words?**
It uses a vector that adds to each word's embedding vector at each position. The model include that vector in the process of learning and it helps with locating the position of words and the distance between them.

**All these vectors seem to add up a big load of math. How do we handle that?**
`Layer Normalization` was introduce in 2016 by three researchers from UofT, and is used to help with normalizing the matrices, hence reducing their dimension in the Transformers.

<img src="http://jalammar.github.io/images/t/transformer_resideual_layer_norm.png" alt="Layer Normalization" width="80%"/>

### Decoder

Almost the same architecture and structure of the Encoder part!

<img src="http://jalammar.github.io/images/t/transformer_decoding_1.gif" alt="Decoder step one" width="80%"/>

* The top encoder sends the attention vectors `K` and `V` to **each decoder** in the decoders stack
* They are used in the `encoder-decoder attention` layer, the extra layer the decoder has compared to the encoder, to help the decoder to focus on the *more important parts* of the input words

**Next steps in the decoder are going to be like this**

<img src="http://jalammar.github.io/images/t/transformer_decoding_2.gif" alt="Decoder steps two and after" width="80%"/>

* The above steps are repeated until a special symbol, `<end of sentence>`, is generated.
* Note that the output of each step is fed to the bottom decoder, very similar to what happened in the encoder part where the input words were fed up
* And similarly again, those bottom inputs in the decoder must be embedded as well

#### Final Linear + Softmax

After the final decoder, how do we convert the numerical vectors into actual words that represent, in this case, the translated sentence?

The linear layer (fully connected NN) simply projects the output vector from the last decoder into a, say, 10000 cell vector where each cell represent a unique English word known by the model.

The softmax layer then projects the score vector into a probability vector, where the position with highest probability determines the actual word outputted by the decoder.

### Actual training

So far, we have talked about how the forward pass of the transformers look like. We also need to train the network, so we need a `loss function` and an `optimization` method for minimizing the loss and increasing the accuracy of the model.


<img src="http://jalammar.github.io/images/t/transformer_logits_output_and_label.png" alt="Desired and untrained output" width="80%"/>

* We are assuming that we have labeled data, hence the desired output
* Those numbers are probabilities, produced by the final Softmax layer in the decoder section
* The desired output differs from the actual output of the untrained model
* We can simply subtract the two vectors, or just use any other distance metric

**This is the output of a trained model:**

<img src="http://jalammar.github.io/images/t/output_trained_model_probability_distributions.png" alt="Trained output" width="80%"/>

## Time for some code!

We could implement transformers using its building blocks one by one and then get it to work by connecting them. However, people have already done it for us in an efficient way. So as we now know how they actually work, we could easily use the existing code to see the performance of the transformers.

For this tutorial, we use [transformer implementation by HuggingFace 🤗](https://huggingface.co/transformers/index.html). It is highly recommended that you use Google Colab to run your code as it requires a lot of downloading and computation to run the following codes.